# Event Driven programming with an example

# 1. Define the Events

* Events as Messages: We define specific EVENT_ constants (like EVENT_CUSTOMER_WALKS_IN). These are the "things that happen."

In [10]:
import time

# --- 1. Define the Events ---
# We'll use simple strings for event names, but in more complex systems,
# these could be custom Event objects with more data.

EVENT_CUSTOMER_WALKS_IN = "customer_walks_in"
EVENT_ORDER_PLACED = "order_placed"
EVENT_COFFEE_READY = "coffee_ready"
EVENT_PAYMENT_RECEIVED = "payment_received"
EVENT_SHIPMENT_ARRIVED = "new_shipment_arrived"

# 2. Define the Central Event Dispatcher

Event Dispatcher (EventDispatcher class): This is the heart of the system.

* It maintains a registry (self.listeners) that maps event names to a list of functions that are interested in that event.
* register_listener(): Staff members "sign up" to listen for specific events.
* fire_event(): When something happens, the dispatcher "fires" the event. It then looks up all functions registered for that event and calls them, passing any relevant data.

In [11]:
# --- 2. The Central Event Dispatcher ---

class EventDispatcher:
    """
    Manages all events and their listeners.
    It's like the nervous system of the coffee shop.
    """
    def __init__(self):
        self.listeners = {} # Dictionary: event_name -> list of listener functions

    def register_listener(self, event_name, listener_function):
        """
        Registers a function to be called when a specific event occurs.
        """
        if event_name not in self.listeners:
            self.listeners[event_name] = []
        self.listeners[event_name].append(listener_function)
        print(f"  [Dispatcher] Registered '{listener_function.__name__}' for '{event_name}'.")

    def fire_event(self, event_name, *args, **kwargs):
        """
        Triggers an event, calling all registered listener functions.
        """
        print(f"\n--- DISPATCHER: Event '{event_name}' fired! ---")
        if event_name in self.listeners:
            for listener in self.listeners[event_name]:
                try:
                    listener(*args, **kwargs) # Call the listener with event data
                except TypeError as e:
                    print(f"  [Dispatcher Error] Listener '{listener.__name__}' failed for '{event_name}': {e}")
        else:
            print(f"  [Dispatcher] No listeners for '{event_name}'.")

# 3. Define the "Staff Members" (Event Listeners/Handlers)

Event Listeners/Handlers (Functions like greet_customer, barista_handle_order):

* These functions represent the "staff members" who are trained to react to certain situations.
* They don't run on their own; they wait to be triggered by the EventDispatcher when a specific event occurs.
* Notice how barista_handle_order itself fires a new event (EVENT_COFFEE_READY) once its task is done. This shows how events can chain.

In [12]:
# --- 3. The "Staff Members" (Event Listeners/Handlers) ---
# These are functions that perform actions when an event is passed to them.

def greet_customer(customer_name):
    print(f"  [Greeter] Welcome, {customer_name}! Please take a look at our menu.")

def barista_handle_order(customer_name, item, size="medium"):
    print(f"  [Barista] Order received from {customer_name}: {size} {item}. Starting preparation...")
    # Simulate making coffee
    time.sleep(1) # Takes a moment to prepare
    print(f"  [Barista] {size} {item} is ready!")
    # Once coffee is ready, the barista fires a new event!
    coffee_shop.dispatcher.fire_event(EVENT_COFFEE_READY, customer_name, item)

def barista_serve_coffee(customer_name, item):
    print(f"  [Barista] Here's your {item}, {customer_name}! Enjoy!")

def cashier_process_payment(customer_name, amount):
    print(f"  [Cashier] Processing payment for {customer_name}: ${amount:.2f}.")
    print(f"  [Cashier] Payment received. Thank you!")
    # Cashier might fire a 'transaction_complete' event here in a real system

def inventory_update_stock(item_name, quantity):
    print(f"  [Inventory] New shipment: {quantity} units of {item_name} received. Updating stock.")
    # In a real system, this would update a database or inventory count

# 4. Integrating the complete Coffee Shop

In [13]:
# --- 4. The Coffee Shop (Putting it all together) ---

class CoffeeShop:
    """
    Our main coffee shop entity that sets up the events and listeners.
    """
    def __init__(self, name):
        self.name = name
        self.dispatcher = EventDispatcher()
        self._setup_listeners()
        print(f"Welcome to {self.name}! The coffee shop is open and ready to serve.")

    def _setup_listeners(self):
        """Registers all staff members to their respective events."""
        print("\n--- Setting up Event Listeners ---")
        self.dispatcher.register_listener(EVENT_CUSTOMER_WALKS_IN, greet_customer)
        self.dispatcher.register_listener(EVENT_ORDER_PLACED, barista_handle_order)
        self.dispatcher.register_listener(EVENT_COFFEE_READY, barista_serve_coffee)
        self.dispatcher.register_listener(EVENT_PAYMENT_RECEIVED, cashier_process_payment)
        self.dispatcher.register_listener(EVENT_SHIPMENT_ARRIVED, inventory_update_stock)

# 5. Simulation/Execution of the Coffee Shop Day

* No Fixed Script: In the "Simulation" part, we don't have a rigid sequence like "first preheat, then mix, then bake." Instead, we just fire_event whenever something happens. The system reacts dynamically.
* Asynchronous Nature (Simulated): Even though Python code runs sequentially, the event-driven structure simulates concurrent activity. The Barista starts making coffee (a simulated delay), but the Cashier can process a payment for another customer before the first coffee is fully ready. The events don't block each other.

In [9]:

# --- Simulation of the Coffee Shop Day ---

print("\n--- Starting the Event-Driven Coffee Shop Simulation ---")

# Create the coffee shop instance, which sets up the event system
coffee_shop = CoffeeShop("The Daily Brew")

# --- Events Firing ---

# Event 1: A customer walks in
coffee_shop.dispatcher.fire_event(EVENT_CUSTOMER_WALKS_IN, "Alice")
time.sleep(0.5)

# Event 2: Alice places an order
coffee_shop.dispatcher.fire_event(EVENT_ORDER_PLACED, "Alice", "Latte", size="large")
time.sleep(0.5) # Simulate time passing while Barista works

# Event 3: Another customer walks in while Barista is busy
coffee_shop.dispatcher.fire_event(EVENT_CUSTOMER_WALKS_IN, "Bob")
time.sleep(0.5)

# Event 4: Bob places an order
coffee_shop.dispatcher.fire_event(EVENT_ORDER_PLACED, "Bob", "Cappuccino")
time.sleep(0.5)

# Event 5: Alice's payment is received (note: order of receipt doesn't strictly follow order of placing)
coffee_shop.dispatcher.fire_event(EVENT_PAYMENT_RECEIVED, "Alice", 5.50)
time.sleep(0.5)

# Event 6: A new shipment of beans arrives
coffee_shop.dispatcher.fire_event(EVENT_SHIPMENT_ARRIVED, "Espresso Beans", 20)
time.sleep(0.5)

# End of simulation
print("\n--- End of Coffee Shop Day Simulation ---")


--- Starting the Event-Driven Coffee Shop Simulation ---

--- Setting up Event Listeners ---
  [Dispatcher] Registered 'greet_customer' for 'customer_walks_in'.
  [Dispatcher] Registered 'barista_handle_order' for 'order_placed'.
  [Dispatcher] Registered 'barista_serve_coffee' for 'coffee_ready'.
  [Dispatcher] Registered 'cashier_process_payment' for 'payment_received'.
  [Dispatcher] Registered 'inventory_update_stock' for 'new_shipment_arrived'.
Welcome to The Daily Brew! The coffee shop is open and ready to serve.

--- DISPATCHER: Event 'customer_walks_in' fired! ---
  [Greeter] Welcome, Alice! Please take a look at our menu.

--- DISPATCHER: Event 'order_placed' fired! ---
  [Barista] Order received from Alice: large Latte. Starting preparation...
  [Barista] large Latte is ready!

--- DISPATCHER: Event 'coffee_ready' fired! ---
  [Barista] Here's your Latte, Alice! Enjoy!

--- DISPATCHER: Event 'customer_walks_in' fired! ---
  [Greeter] Welcome, Bob! Please take a look at our m

# Key Insights

This implementation demonstrates the core principles of Event-Driven Programming, where the flow of control is determined by external occurrences rather than a predefined linear sequence.

# COMPLETED